# Imports and Setups

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
! pip install setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=d131f5abf074326a9c073d7fb2eaa43673a49041393912d4dde617643baf3269
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence

In [3]:
!pip install keybert

  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23767 sha256=565d2b59a86a0bf269521c15c8211e465d7a303ffd7fe5f64668faae8c07633d
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
Successfully built keybert


Lendo o Arquivo

In [1]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [3]:
num_gen = 5
perc = 5
i=5

In [4]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i)+'.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [5]:
base_cru_1p.head()

,texto,label
0,I'm seeing #catfish while I search for a #fish...,1
1,Love these cold winter mornings 😬 best feeling...,1
2,"My professors moustache is so uneven, Mohstash...",0
3,@user y r women considered inferior2men by soc...,1
4,Ma thing,1


In [6]:
n_labels = base_cru_1p.label.nunique()
n_labels

2

In [7]:
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(base_cru_1p)

In [8]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset

In [9]:
train_dataset = sample_dataset(train, label_column="label", num_samples=16)

In [10]:
train_dataset

Dataset({
    features: ['texto', 'label'],
    num_rows: 32
})

In [11]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

def f1_multiclass(labels, preds):
      return f1_score(labels, preds, average='macro')

In [12]:
classifica = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    # use_differentiable_head=True,
    # head_params={"out_features": 6},
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [13]:
trainer = SetFitTrainer(
    model=classifica,
    train_dataset=train_dataset,
    # eval_dataset=eval,
    loss_class=CosineSimilarityLoss,
    # metric=f1_multiclass,
    batch_size=16,
    num_iterations=20, # The number of text pairs to generate for contrastive learning 40
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"texto": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

In [14]:
trainer.train()

Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1280
  Num epochs = 1
  Total optimization steps = 80
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/80 [00:00<?, ?it/s]

In [15]:
pred = classifica(base_cru_1p.texto.values)

In [16]:
f1_multiclass(base_cru_1p.label.values, pred.tolist())

0.6929637526652452

In [17]:
trainer.model._save_pretrained(save_directory="./setfit_output/")

In [18]:
def retorna_word(lista_palavras):
  res = []
  for a in lista_palavras:
    res.append(a[0])
  return res

In [19]:

from keybert import KeyBERT
kw_model = KeyBERT(model = "./setfit_output/")


In [20]:
base_cru_1p['keys'] = base_cru_1p['texto'].progress_apply(lambda x:retorna_word(kw_model.extract_keywords(x, keyphrase_ngram_range=(1, 3), top_n=3, use_mmr=True, stop_words = None)))

100%|██████████| 144/144 [00:09<00:00, 15.03it/s]


In [21]:
base_cru_1p.head()

,texto,label,keys
0,I'm seeing #catfish while I search for a #fish...,1,"[seeing catfish while, my goldfish lovelife, l..."
1,Love these cold winter mornings 😬 best feeling...,1,"[winter mornings best, best feeling everrrrrrr..."
2,"My professors moustache is so uneven, Mohstash...",0,"[professors moustache is, so uneven mohstash, ..."
3,@user y r women considered inferior2men by soc...,1,"[men need women4decision, inferior2men by soci..."
4,Ma thing,1,"[ma thing, thing, ma]"


In [22]:
base_cru_1p['keys_text'] = base_cru_1p['keys'].progress_apply(lambda x: " - ".join(x))
base_cru_1p.head()

100%|██████████| 144/144 [00:00<00:00, 28337.23it/s]


,texto,label,keys,keys_text
0,I'm seeing #catfish while I search for a #fish...,1,"[seeing catfish while, my goldfish lovelife, l...",seeing catfish while - my goldfish lovelife - ...
1,Love these cold winter mornings 😬 best feeling...,1,"[winter mornings best, best feeling everrrrrrr...",winter mornings best - best feeling everrrrrrr...
2,"My professors moustache is so uneven, Mohstash...",0,"[professors moustache is, so uneven mohstash, ...",professors moustache is - so uneven mohstash -...
3,@user y r women considered inferior2men by soc...,1,"[men need women4decision, inferior2men by soci...",men need women4decision - inferior2men by soci...
4,Ma thing,1,"[ma thing, thing, ma]",ma thing - thing - ma


In [23]:
pos_key = {}
for c in base_cru_1p['label'].unique():
  data = base_cru_1p[base_cru_1p['label']==c].copy()
  original = data[['keys_text', 'texto', 'label']].values.tolist()

  pos_key[c] = original

  print(len(pos_key[c]))

73
71


In [24]:
from random import sample

examples = []
generate = []
original = []
labels = []

for c in base_cru_1p['label'].unique():
  for n in range(int(2*num_gen*len(pos_key[c]))):
    if len(pos_key[c]) > 11:
      amostra = sample(pos_key[c],11)
    else:
      amostra = sample(pos_key[c],len(pos_key[c]))

    gen = amostra.pop()

    examples.append(amostra)
    generate.append(gen[0])
    original.append(gen[1])
    labels.append(gen[2])


data_fim = {'examples':examples, 'generate':generate, 'original':original, 'labels':labels}

base_gen = pd.DataFrame.from_dict(data_fim)
base_gen.head()

,examples,generate,original,labels
0,"[[sooo excited about - have pink eye - it, so ...",winter mornings best - best feeling everrrrrrr...,Love these cold winter mornings 😬 best feeling...,1
1,[[you send presents - of the stuff - who live ...,minutes for our - giving us 55 - science depar...,Thank you Harding science department for only ...,1
2,[[seeing catfish while - my goldfish lovelife ...,enough crave salad - round xmas - more funnily...,@user bring it on. Will look forward to the Ea...,1
3,"[[just so lovely - 8ams are - are, 8ams are ju...",priorities intact dc - council has it - glad the,I'm glad the DC Council has it's priorities in...,1
4,[[for two hours - board about dental - wanted ...,lies everything is - everything is fake - cala...,It's all a calamity of lies..everything is fak...,1


In [25]:
base_gen.shape

(1440, 4)

In [26]:
def cria_prompt(ordem, ex, synt):
  prompt = ordem
  for el in ex:
    soma = '\nkeys: '+ el[0]+'\ntweet: '+el[1]+'\n###'
    prompt+= soma

  prompt += '\nkeys: '+ synt+'\ntweet: '
  return prompt



In [27]:
base_gen['prompt'] = base_gen.progress_apply(lambda x: cria_prompt("Generate a tweet using a few keywords. The tweet MUST contain these keywords:", x.examples, x.generate), axis=1)
base_gen.head()

100%|██████████| 1440/1440 [00:00<00:00, 21353.06it/s]


,examples,generate,original,labels,prompt
0,"[[sooo excited about - have pink eye - it, so ...",winter mornings best - best feeling everrrrrrr...,Love these cold winter mornings 😬 best feeling...,1,Generate a tweet using a few keywords. The twe...
1,[[you send presents - of the stuff - who live ...,minutes for our - giving us 55 - science depar...,Thank you Harding science department for only ...,1,Generate a tweet using a few keywords. The twe...
2,[[seeing catfish while - my goldfish lovelife ...,enough crave salad - round xmas - more funnily...,@user bring it on. Will look forward to the Ea...,1,Generate a tweet using a few keywords. The twe...
3,"[[just so lovely - 8ams are - are, 8ams are ju...",priorities intact dc - council has it - glad the,I'm glad the DC Council has it's priorities in...,1,Generate a tweet using a few keywords. The twe...
4,[[for two hours - board about dental - wanted ...,lies everything is - everything is fake - cala...,It's all a calamity of lies..everything is fak...,1,Generate a tweet using a few keywords. The twe...


In [28]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neo-2.7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

In [29]:
def gera_text(modelo, tokens, dev, ppt, temperatura, max_tokens):
  inputs = tokens(ppt, return_tensors="pt").to(dev)
  outputs = modelo.generate(**inputs, max_new_tokens=max_tokens, temperature=temperatura, do_sample =True)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return result

In [30]:
device = "cuda:0"
print(gera_text(model_4bit, tokenizer, device,base_gen['prompt'].iloc[4], 0.25, 20))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generate a tweet using a few keywords. The tweet MUST contain these keywords:
keys: for two hours - board about dental - wanted to
tweet: Because standing by a board about dental hygiene for two hours is how I wanted to spend my birthday..
###
keys: enough crave salad - round xmas - more funnily enough
tweet: @user bring it on. Will look forward to the Easter one more :-))) funnily enough i crave salad and fruit round xmas now!!
###
keys: accidentally spray perfume - haha love when - eye
tweet: Haha love when I accidentally spray perfume in my eye 🙆
###
keys: sunrise google doesn - calendar - how far back
tweet:.@sunrise how far back can I search my google calendar via sunrise? google doesn't have a working search engine on its calendar
###
keys: bit of tit - who wrote that - user the person
tweet: @user The person who wrote that is clearly a bit of a tit
###
keys: our gender struggles - see continuity in - it nice to
tweet: It's nice to see continuity in our gender struggles.
###
keys

In [31]:
device = "cuda:0"
base_gen['gpt_resp'] = base_gen['prompt'].progress_apply(lambda x: gera_text(model_4bit, tokenizer, device, x, 0.25, 20))

  7%|▋         | 105/1440 [04:35<56:03,  2.52s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 31%|███       | 444/1440 [19:36<43:17,  2.61s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 69%|██████▉   | 1000/1440 [44:04<19:14,  2.62s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 83%|████████▎ | 1194/1440 [52:36<10:31,  2.57s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 96%|█████████▌| 1383/1440 [1:00:56<02:34,  2.71s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 1440/144

In [32]:

# base_gen['gpt_resp'] = ''



In [33]:
# arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/base_emoint_intermediario.csv'
# base_gen = pd.read_pickle(arq)
# base_gen.head()

In [34]:
# gera_text(llm, base_gen['prompt'].iloc[33]).split('\n')[0]

In [35]:
# for j in range(base_gen.shape[0]):
#   if j%100==0:
#     print(j)
#   if base_gen['gpt_resp'].iloc[j]== '':
#     ger = gera_text(llm, base_gen['prompt'].iloc[j]).split('\n')[0]
#     base_gen['gpt_resp'].iloc[j] = ger

In [36]:
base_gen.head()

,examples,generate,original,labels,prompt,gpt_resp
0,"[[sooo excited about - have pink eye - it, so ...",winter mornings best - best feeling everrrrrrr...,Love these cold winter mornings 😬 best feeling...,1,Generate a tweet using a few keywords. The twe...,Generate a tweet using a few keywords. The twe...
1,[[you send presents - of the stuff - who live ...,minutes for our - giving us 55 - science depar...,Thank you Harding science department for only ...,1,Generate a tweet using a few keywords. The twe...,Generate a tweet using a few keywords. The twe...
2,[[seeing catfish while - my goldfish lovelife ...,enough crave salad - round xmas - more funnily...,@user bring it on. Will look forward to the Ea...,1,Generate a tweet using a few keywords. The twe...,Generate a tweet using a few keywords. The twe...
3,"[[just so lovely - 8ams are - are, 8ams are ju...",priorities intact dc - council has it - glad the,I'm glad the DC Council has it's priorities in...,1,Generate a tweet using a few keywords. The twe...,Generate a tweet using a few keywords. The twe...
4,[[for two hours - board about dental - wanted ...,lies everything is - everything is fake - cala...,It's all a calamity of lies..everything is fak...,1,Generate a tweet using a few keywords. The twe...,Generate a tweet using a few keywords. The twe...


In [37]:
def separa_resp(gerar, resp):
  fim = resp.split(gerar+'\ntweet:')[1]
  return fim.split('###')[0].replace('\n', '')


In [38]:
base_gen['gen_fim'] = base_gen.progress_apply(lambda x: separa_resp(x.generate, x.gpt_resp), axis=1)
base_gen.head()

100%|██████████| 1440/1440 [00:00<00:00, 43592.60it/s]


,examples,generate,original,labels,prompt,gpt_resp,gen_fim
0,"[[sooo excited about - have pink eye - it, so ...",winter mornings best - best feeling everrrrrrr...,Love these cold winter mornings 😬 best feeling...,1,Generate a tweet using a few keywords. The twe...,Generate a tweet using a few keywords. The twe...,ive been waking up in winter mornings and lov...
1,[[you send presents - of the stuff - who live ...,minutes for our - giving us 55 - science depar...,Thank you Harding science department for only ...,1,Generate a tweet using a few keywords. The twe...,Generate a tweet using a few keywords. The twe...,@science_department 55 minutes for our scien...
2,[[seeing catfish while - my goldfish lovelife ...,enough crave salad - round xmas - more funnily...,@user bring it on. Will look forward to the Ea...,1,Generate a tweet using a few keywords. The twe...,Generate a tweet using a few keywords. The twe...,#RoundXmas is enough crave salad. Round xmas...
3,"[[just so lovely - 8ams are - are, 8ams are ju...",priorities intact dc - council has it - glad the,I'm glad the DC Council has it's priorities in...,1,Generate a tweet using a few keywords. The twe...,Generate a tweet using a few keywords. The twe...,@dc council has it. Glad the council has it.
4,[[for two hours - board about dental - wanted ...,lies everything is - everything is fake - cala...,It's all a calamity of lies..everything is fak...,1,Generate a tweet using a few keywords. The twe...,Generate a tweet using a few keywords. The twe...,Lies everything is fake. Everything is fake...


In [39]:
# arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/base_emoint_intermediario.csv'
# base_gen.to_pickle(arq)

In [40]:
# base_gen['gpt_resp'] = base_gen['prompt'].progress_apply(lambda x: gera_text(llm, x))
# base_gen.head()

In [41]:
exp_lambada = []

limiar = 1.2/n_labels
base_gen = base_gen[base_gen['gen_fim']!="ERRO"].copy()
preds = classifica.predict_proba(base_gen['gen_fim'].to_list()).tolist()
base_gen['preds'] = [preds[i] for i in range(len(preds))]
base_gen['prob_text'] = base_gen.progress_apply(lambda x:(x.preds[x.labels]), axis=1)
# fim = base_gen[base_gen['prob_text']>= limiar].copy()
fim = base_gen.copy()
fim.rename(columns = {'original':'texto', 'gen_fim':'generated', 'labels':'new_label'}, inplace = True)


exp_lambada.append(fim[['texto','generated', 'new_label', 'prob_text']].copy())

100%|██████████| 1440/1440 [00:00<00:00, 39084.19it/s]


In [42]:
exp_lambada[0].shape

(1440, 4)

In [43]:
base_cru_1p.shape

(144, 4)

In [44]:
exp_lambada[0].shape[0]/base_cru_1p.shape[0]

10.0

In [45]:
fim = []

for c in base_cru_1p.label.unique():
  limite = base_cru_1p[base_cru_1p.label==c].copy().shape[0]*num_gen
  gerados = exp_lambada[0][exp_lambada[0]['new_label']==c].copy()
  # gerados = gerados.drop_duplicates(subset = ['generated'])

  gerados = gerados.sort_values(by='prob_text', ascending=False)
  gerados = gerados.head(limite)
  fim.append(gerados)

In [46]:
base_aug_fim = pd.concat(fim)
base_aug_fim.head()

,texto,generated,new_label,prob_text
356,Yeah this is so cool just fucking woke up,!!! Fucking woke up!!! So cool just this is so,1,0.987677
719,Yeah this is so cool just fucking woke up,FUCKING WOKE UP. THIS IS SO COOL. JUST FUCKI...,1,0.983947
238,Yeah this is so cool just fucking woke up,Fucking woke up. So cool just this is so.,1,0.983586
689,Yeah this is so cool just fucking woke up,Fucking woke up. This is so cool.,1,0.982942
37,Yeah this is so cool just fucking woke up,"FUCKING WOKE UP, SO COOL JUST THIS IS SO",1,0.981777


In [47]:
base_aug_fim.shape[0]/base_cru_1p.shape[0]

5.0

In [48]:
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/teste_algo/base_ironic_'+str(perc)+'_gptneokey'+str(i)+'.csv'
base_aug_fim.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')